# Character Text Splitter
`CharacterTextSplitter`는 가장 기본적인 텍스트 분할 도구입니다.  
특정 문자(separator)를 기준으로 텍스트를 나누어 작은 청크(chunk)로 만들어줍니다.

**chunk란?**  
문장을 분석/처리 하기 쉽게 텍스트를 작은 단위로 나눈 조각을 의미합니다.

In [ ]:
# 필요한 라이브러리 설치
%pip install langchain-text-splitters -q

### 1. 기본 사용법

CharacterTextSplitter의 기본 구조는 다음과 같습니다:

**주요 매개변수**
- `separator`: 텍스트를 나눌 기준 문자 (기본값: `"\n\n"`)
- `chunk_size`: 각 청크의 최대 문자 수
- `chunk_overlap`: 인접한 청크 간 겹치는 문자 수
- `length_function`: 텍스트 길이를 측정하는 함수


In [ ]:
from langchain_text_splitters import CharacterTextSplitter

# 예제 1: 기본적인 텍스트 분할
sample_text = """
안녕하세요. 저는 AI 어시스턴트입니다.

오늘은 CharacterTextSplitter에 대해 배워보겠습니다.

이 도구는 텍스트를 작은 조각으로 나누는 데 사용됩니다.

매우 유용한 도구입니다.
"""

# 기본 설정으로 텍스트 분할기 생성
basic_splitter = CharacterTextSplitter(
    chunk_size=30,     # 각 청크 최대 50자
    chunk_overlap=5   # 10자씩 겹침
)

# 텍스트 분할 실행
chunks = basic_splitter.split_text(sample_text)

print(f"\n분할 결과: {len(chunks)}개의 청크 생성")

for i, chunk in enumerate(chunks, 1):
    print(f"청크 {i}: '{chunk}'")
    print(f"길이: {len(chunk)}자\n")


## 2. 다양한 Separator 활용

Separator는 텍스트를 나누는 기준점입니다. 상황에 따라 다른 separator를 사용할 수 있습니다.

### 자주 사용되는 Separator들:
- `"\n\n"`: 문단 단위로 분할 (기본값)
- `"\n"`: 줄 단위로 분할  
- `". "`: 문장 단위로 분할
- `","`: 쉼표 단위로 분할
- `" "`: 단어 단위로 분할


In [ ]:
# 예제 2: 다양한 Separator 비교
# 공통으로 사용할 텍스트 정의
test_text = """인공지능은 현대 기술의 핵심입니다. 머신러닝, 딥러닝, 자연어처리 등 다양한 분야가 있습니다.
최근에는 ChatGPT, GPT-4 등의 대화형 AI가 주목받고 있습니다. 이러한 기술들은 우리 생활을 크게 변화시키고 있습니다."""

문장단위로 분할해봅시다.

In [ ]:
sentence_splitter = CharacterTextSplitter(
    separator=". ",     # 마침표 + 공백으로 문장 구분
    chunk_size=60,      # 적당한 크기로 설정
    chunk_overlap=10    # 문맥 연결을 위한 겹침
)

sentence_chunks = sentence_splitter.split_text(test_text)

print(f"결과: {len(sentence_chunks)}개 청크 생성")
print(f"설정: 최대 60자, 10자 겹침\n")

for i, chunk in enumerate(sentence_chunks, 1):
    print(f"청크 {i} ({len(chunk)}자): '{chunk}'")
    print()


## 3. Chunk Size와 Chunk Overlap의 효과

### Chunk Size (청크 크기)
- 각 텍스트 조각의 최대 길이를 결정
- 너무 작으면: 문맥이 잘릴 수 있음
- 너무 크면: 처리 효율성이 떨어질 수 있음
    - 임베딩 정확도가 낮아짐
    - 토큰수 증가

### Chunk Overlap (청크 겹침)
- 인접한 청크들 간의 겹치는 부분
- 문맥 연속성을 유지하는 데 중요
- 일반적으로 chunk_size의 10-20% 정도 설정


 *chunk_size = 300 ~ 800, chunk_overlap = 50 ~ 100 가 효율적이라고 알려져 있음*

이번에는 `File`을 통해 파일을 읽어와서 `TextSplitter`로 분할해보겠습니다.

In [ ]:
# data/appendix-keywords.txt 파일을 열어서 f라는 파일 객체를 생성합니다.
with open("./data/training_log_all.txt") as f:
    file = f.read()  # 파일의 내용을 읽어서 file 변수에 저장합니다.

print(file[:500])

In [ ]:

sentence_splitter = CharacterTextSplitter(
    separator=" ",
    chunk_size=100,      # 적당한 크기로 설정
    chunk_overlap=20    # 문맥 연결을 위한 겹침
)

sentence_chunks = sentence_splitter.split_text(file)

print(f"결과: {len(sentence_chunks)}개 청크 생성")

for i, chunk in enumerate(sentence_chunks, 1):
    print(f"청크 {i} ({len(chunk)}자): '{chunk}'")
    print()


## 4. Document 객체 활용

CharacterTextSplitter는 두 가지 메서드를 제공합니다:

1. **`split_text(text)`**: 단순 문자열 리스트 반환
2. **`create_documents(texts, metadatas)`**: Document 객체 리스트 반환

Document 객체는 메타데이터를 포함할 수 있어 더 풍부한 정보를 저장할 수 있습니다.


In [ ]:
# 예제 4: Document 객체와 메타데이터 활용
document_texts = [
    "Python은 간단하고 읽기 쉬운 프로그래밍 언어입니다. 데이터 분석, 웹 개발, AI 개발 등에 널리 사용됩니다.",
    "JavaScript는 웹 개발의 핵심 언어입니다. 브라우저와 서버 양쪽에서 실행할 수 있습니다."
]

# 각 문서에 대한 메타데이터 정의
document_metadata = [
    {"source": "text문서"},
    {"source": "text문서"},
]
# Document 객체 생성
documents = sentence_splitter.create_documents(document_texts, document_metadata)

print(documents[0])
